In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

2023-06-09 21:22:46.611942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 21:22:47.691664: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class Backbone(nn.Module):
    def __init__(self, in_chan=3, out_dim=512):
        super(Backbone, self).__init__()
        
        modules = list(models.resnet18(weights=None).children())[:-1]
        self.model = nn.Sequential(*modules)
        self.linear = nn.Linear(512, 200)
        
    def forward(self, x):
        embed = self.model(x).squeeze()
#         x = self.linear(embed)
        return embed


In [7]:
# Define the network
class ClassifierNet(nn.Module):
    def __init__(self, out_dim, num_classes):
        super(ClassifierNet, self).__init__()
        self.backbone = Backbone(3, out_dim)

        # Add a linear layer on top
        self.linear = nn.Linear(out_dim, num_classes)

        
    def forward(self, x):
        x = self.backbone(x)
        x = self.linear(x)

        return x

In [8]:
# Initialize the dataset and dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to (224, 224)
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize using ImageNet statistics  
])

train_dataset = datasets.Flowers102(root='./data', download=False, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

test_dataset = datasets.Flowers102(root='./data', download=True, transform=transform, split="test")
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [9]:
# Initialize the network
num_classes = 102
model = ClassifierNet( out_dim=512, num_classes=num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# load in the weights
backbone_weights = torch.load('pet_resnet_dc_e5_c200_s8000.pth', map_location=torch.device(device))
model.backbone.load_state_dict(backbone_weights)


# # freeze the backbone so that the gradients of the final layer don't drastically disturb it
# for param in model.backbone.parameters():
#     param.requires_grad = False


<All keys matched successfully>

In [6]:
# # unfreeze the backbone
# for param in model.backbone.parameters():
#     param.requires_grad = True
    

In [10]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Initialize TensorBoard writer
writer = SummaryWriter("./logs/c_dc_resnet_e5_c200_s8000")

num_epochs = 75
model.to(device)

# Training loop

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Log the loss
    writer.add_scalar('Loss/train', running_loss / len(train_dataloader), epoch)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}')
    
    # Evaluation loop
    model.eval()  # Set the network to evaluation mode
    correct = 0
    total = 0
    
    if epoch==10:
        for name, param in model.backbone.named_parameters():
            if not param.requires_grad:
                print(name, param.data[0])
            break
    
    
    with torch.no_grad():
        
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            
            # Get the predicted class labels
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    
    # Log test accuracy to TensorBoard
    writer.add_scalar('Accuracy/Test', accuracy, epoch)
    
    writer.flush()

# Close TensorBoard writer
writer.close()

Epoch [1/75], Loss: 4.6368
Test Accuracy: 1.02%
Epoch [2/75], Loss: 4.5625
Test Accuracy: 2.41%
Epoch [3/75], Loss: 4.3353
Test Accuracy: 2.70%
Epoch [4/75], Loss: 4.0497
Test Accuracy: 4.85%
Epoch [5/75], Loss: 3.8489
Test Accuracy: 5.92%
Epoch [6/75], Loss: 3.6864
Test Accuracy: 7.22%
Epoch [7/75], Loss: 3.5651
Test Accuracy: 6.52%
Epoch [8/75], Loss: 3.3747
Test Accuracy: 9.45%
Epoch [9/75], Loss: 3.1617
Test Accuracy: 9.04%
Epoch [10/75], Loss: 2.9048
Test Accuracy: 10.64%
Epoch [11/75], Loss: 2.7921
Test Accuracy: 10.28%
Epoch [12/75], Loss: 2.5583
Test Accuracy: 15.35%
Epoch [13/75], Loss: 2.2928
Test Accuracy: 15.12%
Epoch [14/75], Loss: 2.2320
Test Accuracy: 14.23%
Epoch [15/75], Loss: 1.9045
Test Accuracy: 16.67%
Epoch [16/75], Loss: 1.7130
Test Accuracy: 14.26%
Epoch [17/75], Loss: 1.4121
Test Accuracy: 17.63%
Epoch [18/75], Loss: 1.1051
Test Accuracy: 17.19%
Epoch [19/75], Loss: 0.8576
Test Accuracy: 17.56%
Epoch [20/75], Loss: 0.7000
Test Accuracy: 17.81%
Epoch [21/75], Los

KeyboardInterrupt: 